1. d
5. g

Math 5366 - Data Mining 2, Tarleton State University, Spring 2018, Dr. Scott Cook

Due Feb 1, 2018

One key fact about Artificial Neural Networks is that they are a universal approximator - they can mimic the behavior of any arbitary function.  Let's explore this in the simple case of functions from $R^1 \to R^1$.

Assumptions for the hwk set:
- All nodes have their own bias (in other words - not 'one bias for the entire layer)
- Input layer uses the identity activation function


1. Use the slightly updated starter code below.  Write forward_propagate(), backward_propagate(), descend_gradient().  Use https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/ to make certain your code works right.

4. Now let's approximate $f(x)=e^{-x^2}$.  Let $X$ be 100 uniformally random numbers between -3 and 3.  Let $Y=f(x)$.  So we have a dataset with $n=100$ obervations, $p=1$ continuous features, and $q=1$ continuous targets.  Use the ANN code we wrote by hand to fit an ANN with 1 hidden layer with 2 nodes using the sigmoid activtion function at each layer except the input.
 - For consistency, use rnd = np.random.RandomState(seed=42)
 - Draw your observations first feat = np.sort(rnd.uniform(-3,3,100)).reshape(-1,1).
 - Pick initial weights rnd.rnd(0,1)
 - Pick initial bias rnd.rnd(0,1)
 - At each cycle, record loss / n.  Do enough forward-back-learn cycles to get an ANN that approximates the function well.  Let's set a cutoff of loss / n < 1e-4.
 - Plot the final ANN approximation on top of the true function.
 - Plot loss / n vs step
 
7. Repeat this process using different network topologies (#layers and #nodes) and different activation function combinations. Each time, record both # steps and time needed to reach the cutoff.  Try to find the ANN that get a good approximator as efficiently as possible.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import itertools as it

# common activation functions and their derivatives
def id(x):
    return x

def id_deriv(x):
    return float(1)

def relu(x):
    x[x<0] = 0
    return x

def relu_deriv(x):
    return (x>0).astype(float)

def tanh(x):
    return np.tanh(x)

def tanh_deriv(x):
    return 1-tanh(x)**2

def arctan(x):
    return np.arctan(x)
  
def arctan_deriv(x):
    return 1 / (1 + x**2)

def sigmoid(x):
    y = np.zeros_like(x)
    idx = x > -700
    y[idx] = 1 / (1 + np.exp(-1*x[idx]))
    return y

def sigmoid_deriv(x):
    y = sigmoid(x)
    return y * (1 - y)

def loss(e):    
    return (e**2).sum() / 2

def loss_deriv(e):
    return e

def pre_allocate():
    ## pre-allocate
    ## X[h,i,j] = input to node j of layer h for observation i; X[h] has shape n x nodes[h]
    X = [np.zeros(shape=[n,p]) for p in nodes]

    ## Y[h,i,j] = output from node j of layer h for observation i; Y[h] has shape n x nodes[h]
    Y = X.copy()

    ## B[h,j] = bias into node j of layer h; B[h] has shape nodes[h]
    B = [rnd.rand(p) for p in nodes]
    B[0] *= 0.0

    ## W_sh[h] = (nodes[h], nodes[h+1])
    W_sh = [(i,o) for (i,o) in zip(nodes[:-1],nodes[1:])]

    ## W[h,j,k] = weight of edge from node j of layer h to node k of layer h+1; W[h] has shape nodes[h] x nodes[h+1]
    W = [rnd.rand(*ws) for ws in W_sh]

    ## E has shape Y[-1]
    E = Y[-1].copy()
    
    ## DLDX[h,i,j] = partial derivative of loss wrt X[h,i,j]; DLDX[h] has shape n x nodes[h]
    DLDX = X.copy()

    ## DLDY[h,i,j] = partial derivative of loss wrt Y[h,i,j]; DLDY[h] has shape n x nodes[h]
    DLDY = X.copy()

    ## DLDB[h,i,j] = partial derivative of loss wrt B[h,j]; DLDB[h] has shape nodes[h]
    DLDB = [np.zeros_like(b) for b in B]
    
    ## DLDW[h,j,k] = partial derivative of loss wrt W[h,j,k]; DLDW[h] has shape nodes[h] x nodes[h+1]
    DLDW = [np.zeros_like(w) for w in W]

    return X, Y, B, W, E, DLDX, DLDY, DLDB, DLDW

## The scaling below is easier with sklearn.preprocessing.  But I wanted to write a purely Numpy algorithm.
def minmaxscale(x, a=0, b=1):
    m = x.min()
    r = x.max() - m
    if r == 0:
        r = 1
    y = (x - m) / r * (b - a) + a
    return y


def fit_ANN():
    global X, Y, B, W, E, DLDX, DLDY, DLDB, DLDW
    loss_hist = []
    for step in range(max_steps):
        X[0] = feat_sc.copy()
        forward_propagate()
        pred_sc = Y[-1].copy()

        E = true_sc - pred_sc  # note: choice of t-p vs p-t may impact the +/- sign in descend_gradient
        L = loss(E) / n
        loss_hist.append(L)

        if loss_hist[-1] < 1e-4:
            break

        backward_propagate()
        descend_gradient()
    return loss_hist


## functions for pieces of the neural network training
def forward_propagate():
    global X, Y

def backward_propagate():
    global DLDX, DLDY, DLDB, DLDW

def descend_gradient():
    global B, W


In [ ]:
## example from https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/, use sigmoid, loss=(e**2)/2 in loss, loss_deriv=e
rnd = np.random.RandomState(42)

feat = np.array([.05,.1]).reshape(1,-1)
true = np.array([.01,.99]).reshape(1,-1)

n, p = feat.shape
m, q = true.shape
if m != n:
    raise Exception('feat and true must have the same number of rows')


hidden_nodes = [2]
max_steps = 1
learn_rate = 0.5

nodes = [p] + hidden_nodes + [q]
layers = len(nodes)

## set activation functions
activation = [sigmoid for l in range(layers)]
activation_deriv = [sigmoid_deriv for l in range(layers)]
activation[0] = id
activation_deriv[0] = id_deriv
# activation[-1] = id
# activation_deriv[-1] = id_deriv


## Usually should scale data to (0,1) during preprocessing.
## Skip for now  to make numbers exactly match Mazur.
feat_sc = feat.copy()
true_sc = true.copy()

X, Y, B, W, E, DLDX, DLDY, DLDB, DLDW = pre_allocate()

## example from https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/, use sigmoid, loss=(e**2)/2 in loss, loss_deriv=e
W = [np.array([[0.15, 0.25], [0.20, 0.30]]), np.array([[0.40, 0.50], [0.45, 0.55]])]
B = [np.array([0.0, 0.0]), np.array([0.35, 0.35]), np.array([0.60, 0.60])]

loss_hist = fit_ANN()     
# %timeit fit_ANN()

for l in range(layers):
    print("layer = {}".format(l))
    print("X")
    print(X[l])
    print("Y")
    print(Y[l])
    print("B")
    print(B[l])
    print("W")
    if l < layers-1:
        print(W[l])
    print()